In [3]:
from io import IncrementalNewlineDecoder
import torch
import torch.autograd as tgrad
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import pandas as pd
%matplotlib inline
     

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.set_default_device(DEVICE)     

## Call Option Pricing

In [6]:
S = torch.Tensor([80]).requires_grad_()
t = torch.Tensor([0]).requires_grad_()
sigma = torch.Tensor([0.3]).requires_grad_()
r = torch.Tensor([0.05]).requires_grad_()
K = torch.Tensor([70])
T = torch.Tensor([1])
t2m = T-t

d1 = (torch.log(S / K) + (r + 0.5 * sigma**2) * t2m)/(sigma * torch.sqrt(t2m))
d2 = d1 - sigma * torch.sqrt(t2m)

In [9]:
N0 = lambda value: 0.5 * (1 + torch.erf((value/2**0.5))) #normal cdf
Nd1 = N0(d1)
Nd2 = N0(d2)
C = S* Nd1 - K* Nd2 *torch.exp(-r*t2m)
print("Option Price:", C.item())

Option Price: 17.014957427978516


### Differential Equation

In [11]:
# First Derivative of Call option pricing function wrt time
dCdt = tgrad.grad(C, t, create_graph=True, only_inputs=True)[0]
dCdt

tensor([-5.8385], grad_fn=<NegBackward0>)

In [12]:
# First Derivative of Call option pricing function wrt stock price
dCdS = tgrad.grad(C, S, create_graph=True, only_inputs=True)[0]

# Second Derivative of Call option pricing function wrt stock price
d2CdS2 = tgrad.grad(dCdS, S, create_graph=True, only_inputs=True)[0]

# First Derivative of Call option pricing function wrt volatility
dCdvol = tgrad.grad(C, sigma, create_graph=True, only_inputs=True)[0]
dCdS,d2CdS2,dCdvol

(tensor([0.7769], grad_fn=<AddBackward0>),
 tensor([0.0124], grad_fn=<AddBackward0>),
 tensor([23.8776], grad_fn=<AddBackward0>))

In [13]:
# Evaluate differential Equation (approx 0)
dCdt.item() + ((0.5*sigma**2) * (S**2)*d2CdS2.item() )+ r*S*dCdS.item() - r*C

tensor([1.7881e-07], grad_fn=<SubBackward0>)